In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [ ]:
!pip install pymupdf


In [ ]:
import fitz  # PyMuPDF
import os

def count_text_tokens(text):
    """Counts the number of tokens in a text string based on word count."""
    # A simple word split to approximate token count
    words = text.split()
    return len(words)

def extract_text_from_pdf(pdf_file_path):
    """Extracts text from a PDF file."""
    try:
        doc = fitz.open(pdf_file_path)
        text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text("text")  # Extract text from each page
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

def count_pdf_tokens(pdf_file_path):
    """Counts tokens in a PDF file."""
    text = extract_text_from_pdf(pdf_file_path)
    return count_text_tokens(text)

def count_tokens_in_pdf_directory(directory_path):
    """Counts total tokens for all PDF files in a directory."""
    total_tokens = 0
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            print(f"Processing file: {filename}")
            file_tokens = count_pdf_tokens(file_path)
            total_tokens += file_tokens
            print(f"Tokens for {filename}: {file_tokens}")

    print(f"Total tokens for all files in the directory: {total_tokens}")
    return total_tokens

# Example usage:
directory_path = "/content/Data/"
total_tokens = count_tokens_in_pdf_directory(directory_path)+


In [ ]:
import os
import json
import base64

def count_text_tokens(text):
    """Counts the number of tokens in a text string based on word count."""
    # A simple word split to approximate token count
    words = text.split()
    return len(words)

def estimate_base64_image_tokens(base64_string):
    """Estimates token cost for a base64-encoded image."""
    try:
        decoded_image = base64.b64decode(base64_string)
        image_size_bytes = len(decoded_image)
        tokens = image_size_bytes / 50  # Adjust the divisor as needed
        return int(tokens)
    except Exception as e:
        print(f"Error decoding base64: {e}")
        return 0

def count_json_tokens(json_file_path, include_images=False):
    """Counts tokens in a JSON file, optionally including base64 images."""
    try:
        with open(json_file_path, "r") as f:
            data = json.load(f)

        # Convert the entire JSON content to a string and count tokens
        json_string = json.dumps(data)
        json_tokens = count_text_tokens(json_string)

        image_tokens = 0
        if include_images:
            if isinstance(data, dict):
                for value in data.values():
                    if isinstance(value, str) and value.startswith("data:image/"):
                        image_tokens += estimate_base64_image_tokens(value.split(',')[1])
            elif isinstance(data, list):
                for item in data:
                    if isinstance(item, dict):
                        for value in item.values():
                            if isinstance(value, str) and value.startswith("data:image/"):
                                image_tokens += estimate_base64_image_tokens(value.split(',')[1])

        return json_tokens + image_tokens

    except FileNotFoundError:
        print(f"Error: File not found: {json_file_path}")
        return 0
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON in file: {json_file_path}")
        return 0

def count_tokens_in_directory(directory_path, include_images=False):
    """Counts total tokens for all JSON files in a directory."""
    total_tokens = 0
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)
            print(f"Processing file: {filename}")
            file_tokens = count_json_tokens(file_path, include_images)
            total_tokens += file_tokens
            print(f"Tokens for {filename}: {file_tokens}")

    print(f"Total tokens for all files in the directory: {total_tokens}")
    return total_tokens

# Example usage:
directory_path = "/content/Data/Book1/Text"
directory_path2 = "/content/Data/Book1/Img"

include_images = True  # Set to False if you don't want to include base64 images

total_tokens =  count_tokens_in_directory(directory_path, include_images) + count_tokens_in_directory(directory_path2, include_images)


In [ ]:
#START OF RAG GIG NO GAG
!pip install langchain supabase sentence-transformers torch google-generativeai python-dotenv tqdm numpy multipart torch pip  langchain-community  InstructorEmbedding

In [ ]:
import os
import json
import logging
from abc import ABC, abstractmethod
from pathlib import Path
from typing import Optional, Dict, Any
from PIL import Image
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from supabase import create_client, Client
from google.colab import userdata

In [ ]:
# Initialize with Colab secrets
SUPABASE_URL = userdata.get('SUPABASE_URL')
SUPABASE_KEY = userdata.get('SUPABASE_ANON_KEY')
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
import os
import json
import logging
from abc import ABC, abstractmethod
from pathlib import Path
from typing import Optional, Dict, Any, List
from PIL import Image
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from supabase import create_client, Client
import uuid

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------- Interfaces ----------------------------
class SummaryGenerator(ABC):
    @abstractmethod
    def generate_text_summary(self, text: str) -> Optional[str]:
        pass

    @abstractmethod
    def generate_image_summary(self, image_path: str) -> Optional[str]:
        pass

class EmbeddingGenerator(ABC):
    @abstractmethod
    def generate_embedding(self, text: str) -> Optional[list]:
        pass

class ContentProcessor(ABC):
    @abstractmethod
    def process(self, base_path: str, course_id: str) -> None:
        pass

# ---------------------------- Implementations ----------------------------
class GeminiSummaryGenerator(SummaryGenerator):
    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)
        self.text_model = genai.GenerativeModel('gemini-2.0-flash')
        self.vision_model = genai.GenerativeModel('gemini-2.0-flash')

    def generate_text_summary(self, text: str) -> Optional[str]:
        try:
            response = self.text_model.generate_content(
                f"Generate a highly detailed and long summary of the given text, ensuring that each response reaches the maximum possible token limit of 8192 tokens without exceeding it. The complete summary must contain a minimum of 32,700 words. If necessary, generate the summary in coherent, self-contained chunks while maintaining logical flow between sections. Avoid unnecessary elaboration or repetition. Directly provide the summary without any introductory phrases, headings, or indications like Here is the summary. Exclude any practice sets, questions, exercises, references to specific pages, or metadata from the summary. Focus entirely on providing a comprehensive, exhaustive, and information-rich summary that fully encapsulates all details of the original text. Do not shorten or simplify any concepts—retain the full depth and complexity of the content. If the summary exceeds 8192 tokens, stop at a natural breakpoint and indicate that additional content will follow in the next response. Ensure continuity between chunks without repeating previously covered information.{text}",
                  generation_config=genai.types.GenerationConfig(
                  max_output_tokens=8192,
                  temperature=0.3,
                  top_p=0.9,
                  top_k=40)
            )
            return response.text
        except Exception as e:
            logger.error(f"Text summary error: {e}")
            return None

    def generate_image_summary(self, image_path: str) -> Optional[str]:
        try:
            img = Image.open(image_path)
            response = self.vision_model.generate_content(
                ["Describe this image concisely:", img]
            )
            return response.text
        except Exception as e:
            logger.error(f"Image summary error: {e}")
            return None

class InstructorEmbeddingGenerator(EmbeddingGenerator):
    def __init__(self, model_name: str = "hkunlp/instructor-xl"):
        self.model = SentenceTransformer(model_name)

    def generate_embedding(self, text: str) -> Optional[list]:
        try:
            instruction = "Represent the summary for retrieval: "
            return self.model.encode([instruction + text])[0].tolist()
        except Exception as e:
            logger.error(f"Embedding error: {e}")
            return None

class SupabaseStorageClient:
    def __init__(self, url: str, key: str):
        self.client = create_client(url, key)

    def insert_record(self, table: str, data: Dict[str, Any]) -> bool:
        try:
            self.client.table(table).insert(data).execute()
            return True
        except Exception as e:
            logger.error(f"Supabase insert error: {e}")
            return False

# ---------------------------- Processors ----------------------------
class TextContentProcessor(ContentProcessor):
    def __init__(
        self,
        summary_generator: SummaryGenerator,
        embedding_generator: EmbeddingGenerator,
        storage_client: SupabaseStorageClient
    ):
        self.summary_generator = summary_generator
        self.embedding_generator = embedding_generator
        self.storage_client = storage_client

    def process(self, base_path: str, course_id: str) -> None:
        # Look for JSON files directly in the base directory
        base_dir = Path(base_path)
        if not base_dir.exists():
            logger.warning(f"Base directory not found: {base_dir}")
            return

        # Process all JSON files in the base directory and subdirectories
        for text_file in base_dir.glob("**/*.json"):
            try:
                with open(text_file, 'r') as f:
                    data = json.load(f)
                    # Handle both formats: direct content string or content in a field
                    content = data if isinstance(data, str) else data.get('content', '')

                    if not content:
                        continue

                    summary = self.summary_generator.generate_text_summary(content)
                    if not summary:
                        continue

                    embedding = self.embedding_generator.generate_embedding(summary)
                    if not embedding:
                        continue

                    # Generate a unique chunk ID using the file name or a UUID if needed
                    chunk_id = text_file.stem

                    record = {
                        'course_id': course_id,
                        # 'chunk_id': chunk_id,
                        'content': content,
                        'content_summary': summary,
                        'summary_vector': embedding
                    }

                    if not self.storage_client.insert_record('material_text', record):
                        logger.error(f"Failed to insert text record: {chunk_id}")
                    else:
                        logger.info(f"Successfully processed text file: {text_file}")
            except Exception as e:
                logger.error(f"Error processing text file {text_file}: {e}")

class ImageContentProcessor(ContentProcessor):
    def __init__(
        self,
        summary_generator: SummaryGenerator,
        embedding_generator: EmbeddingGenerator,
        storage_client: SupabaseStorageClient
    ):
        self.summary_generator = summary_generator
        self.embedding_generator = embedding_generator
        self.storage_client = storage_client
        # Expanded list of image formats
        self.supported_formats = ['.png', '.jpg', '.jpeg', '.jb2', '.bmp', '.tiff', '.gif']

    def process(self, base_path: str, course_id: str) -> None:
        # Look for images directly in the base directory and subdirectories
        base_dir = Path(base_path)
        if not base_dir.exists():
            logger.warning(f"Base directory not found: {base_dir}")
            return

        # Process all image files in base directory and subdirectories
        for image_file in base_dir.glob("**/*"):
            if image_file.suffix.lower() not in self.supported_formats:
                continue

            try:
                # Try to open the image first to check if PIL can handle it
                try:
                    Image.open(str(image_file)).verify()  # Verify the image can be opened
                except Exception as img_err:
                    logger.warning(f"Skipping unsupported image format: {image_file} - Error: {img_err}")
                    continue

                summary = self.summary_generator.generate_image_summary(str(image_file))
                if not summary:
                    continue

                embedding = self.embedding_generator.generate_embedding(summary)
                if not embedding:
                    continue

                chunk_id = image_file.stem
                record = {
                    'course_id': course_id,
                    # 'chunk_id': chunk_id,  # This line is commented out
                    'content_path': str(image_file),
                    'content_summary': summary,
                    'summary_vector': embedding
                }

                if not self.storage_client.insert_record('material_image', record):
                    logger.error(f"Failed to insert image record: {chunk_id}")
                else:
                    logger.info(f"Successfully processed image: {image_file}")
            except Exception as e:
                logger.error(f"Error processing image {image_file}: {e}")

# ---------------------------- Client Code ----------------------------
class ContentProcessingPipeline:
    def __init__(
        self,
        processors: List[ContentProcessor],
        storage_client: SupabaseStorageClient
    ):
        self.processors = processors
        self.storage_client = storage_client

    def run(self, base_path: str, course_id: str) -> None:
        if not Path(base_path).exists():
            logger.error(f"Base path does not exist: {base_path}")
            # Create the directory if it doesn't exist
            Path(base_path).mkdir(parents=True, exist_ok=True)
            logger.info(f"Created directory: {base_path}")

        logger.info(f"Starting processing for course: {course_id}")
        for processor in self.processors:
            processor.process(base_path, course_id)
        logger.info(f"Completed processing for course: {course_id}")

# ---------------------------- Factory ----------------------------
class ProcessorFactory:
    @staticmethod
    def create_pipeline(
        supabase_url: str,
        supabase_key: str,
        gemini_api_key: str
    ) -> ContentProcessingPipeline:
        # Initialize dependencies
        storage_client = SupabaseStorageClient(supabase_url, supabase_key)
        summary_generator = GeminiSummaryGenerator(gemini_api_key)
        embedding_generator = InstructorEmbeddingGenerator()

        # Create processors
        text_processor = TextContentProcessor(
            summary_generator, embedding_generator, storage_client
        )
        image_processor = ImageContentProcessor(
            summary_generator, embedding_generator, storage_client
        )

        return ContentProcessingPipeline(
            processors=[text_processor,
                        image_processor],
            storage_client=storage_client
        )

# ---------------------------- Usage ----------------------------
def main():
    # Initialize with secrets (in production, use environment variables)
    pipeline = ProcessorFactory.create_pipeline(
        supabase_url=userdata.get('SUPABASE_URL'),
        supabase_key=userdata.get('SUPABASE_ANON_KEY'),
        gemini_api_key=userdata.get('GEMINI_API_KEY')
    )

    # Run processing - adjust the path to your actual content location
    pipeline.run(
        base_path="/content/Data/Book1/",
        course_id="2e51c619-03d2-4b36-90e5-cbd9e9ad0baa"
    )

if __name__ == "__main__":
    main()

ERROR:__main__:Text summary error: HTTPConnectionPool(host='localhost', port=36159): Read timed out. (read timeout=600.0)
ERROR:__main__:Text summary error: HTTPConnectionPool(host='localhost', port=36159): Read timed out. (read timeout=600.0)
ERROR:__main__:Text summary error: HTTPConnectionPool(host='localhost', port=36159): Read timed out. (read timeout=600.0)
ERROR:__main__:Text summary error: HTTPConnectionPool(host='localhost', port=36159): Read timed out. (read timeout=600.0)
ERROR:__main__:Text summary error: HTTPConnectionPool(host='localhost', port=36159): Read timed out. (read timeout=600.0)
